<h1 align="center"> Introdução ao Processamento de Linguagem Natural (PLN) Usando Python </h1>
<h3 align="center"> Professor Fernando Vieira da Silva MSc.</h3>

<h2> Técnicas para Pré-Processamento - Parte 2</h2>

<p>Uma vez que o texto já foi devidamente tratado, removendo stopwords e pontuações, e aplicando stemming ou lemmatization, agora precisamos contar a frequência das palavras (ou n-grams) que utilizaremos em seguida como atributos para as técnicas de aprendizado de máquina.</p>

<b>1. TF-IDF (Term Frequency - Inverse Document Frequency)</b>

<p><b>Term Frequency:</b> um termo que aparece muito em um documento, tende a ser um termo importante. Em resumo, divide-se o número de vezes em que um termo apareceu pelo maior número de vezes em que algum outro termo apareceu no documento.</p>

tf<sub>wd</sub> = f<sub>wd</sub> / m<sub>wd</sub>

onde:<br>
f<sub>wd</sub> é o número de vezes em que o termo <i>w</i> aparece no documento <i>d</i>.<br>
m<sub>wd</sub> é o maior valor de f<sub>wd</sub> obtido para algum termo do documento <i>d</i><br>

<p><b>Inverse Document Frequency:</b> um termo que aparece em poucos documentos pode ser um bom descriminante. Obtem-se dividindo o número de documentos pelo número de documentos em que o termo aparece.</p>

idf<sub>w</sub> = log<sub>2</sub>(n / n<sub>w</sub>)

onde:<br>
n é o número de documentos no corpus
n<sub>w</sub> é o número de documentos em que o termo <i>w</i> aparece.

Na prática, usa-se:

tf-idf = tf<sub>wd</sub> * idf<sub>w</sub>

Podemos calcular o TF-IDF de um corpus usando o pacote <b>scikit-learn</b>. Primeiramente, vamos abrir novamente o texto de Hamlet e armazenar as sentenças em uma ndarray do numpy (como se cada sentença fosse um documento do corpus):

In [1]:
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize

hamlet_raw = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')

sents = sent_tokenize(hamlet_raw)

hamlet_np = np.array(sents)

print(hamlet_np.shape)


(2355,)


<p>Agora vamos definir uma função para tokenização pelo scikit-learn.</p>

In [2]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas
    
    

E essa função será chamada pelo objeto TfidfVectorizer

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

hamlet_raw = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')

sents = sent_tokenize(hamlet_raw)

hamlet_np = np.array(sents)

tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)

tfs = tfidf_vectorizer.fit_transform(hamlet_np)

print(tfs.shape)

(2355, 4305)


In [5]:
print([k for k in tfidf_vectorizer.vocabulary_.keys()][:20])

['tragedie', 'hamlet', 'william', 'shakespeare', '1599', 'actus', 'primus', 'scoena', 'prima', 'enter', 'barnardo', 'francisco', 'two', 'centinels', "'s", 'fran', 'nay', 'answer', 'stand', 'vnfold']


In [6]:
print(tfs[:50,:50])

  (0, 17)	0.40984054295
  (4, 5)	1.0
  (12, 6)	0.255695651027
  (13, 6)	0.241831885737
  (22, 5)	0.58657679876
  (27, 0)	0.226773205328
  (29, 5)	0.284581755506
  (37, 5)	0.264020196593
  (37, 0)	0.236027784598
  (39, 6)	0.138754555877
  (40, 13)	0.352972670444
  (41, 25)	0.314756261552
  (43, 5)	0.144160249461
  (46, 5)	0.322996303502


<b>2. TF-IDF de N-gramas</b>

Opcionalmente, podemos obter os atributos tf-idf de n-grams, combinando as classes CountVectorizer e TfidfTransformer. Em nosso exemplo, vamos utilizar apenas trigramas:

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(ngram_range=(3,3))

n_gram_counts = count_vect.fit_transform(hamlet_np)

tfidf_transformer = TfidfTransformer()

tfs_ngrams = tfidf_transformer.fit_transform(n_gram_counts)

print(tfs_ngrams.shape)

(2355, 23483)


<b>3. Redução de Dimensionalidade</b>

<p>A transformação do corpus em atributos contendo as frequências TF-IDF em geral resultará numa ndarray bastante esparsa, ou seja, com muitas dimensões. Porém, além de isso tornar o treinamento de algoritmos mais demorado e custoso (computacionalmente falando), muitas dessas dimensões provavelmente são pouco representativas ou mesmo podem causar ruído durante o treinamento. Para resolver esse problema, podemos aplicar uma técnica de redução de dimensionalidade simples chamada <b>Singular Value Decomposition (SVD)</b>. 

<p>Essa técnica transformará os vetores da matriz original, rotacionando e escalando-os, resultando em novas representações. A redução de dimensionalidade é feita ao manter apenas as <i>k</i> dimensões mais representativas que escolhermos. Outra vantagem dessa técnica é que as dimensões originais são, de certa forma, "combinadas", o que resulta em uma nova forma de representar a combinação de termos. No contexto de PLN, essa técnica é conhecida como <b>Latent Semantic Analysis (LSA)</b></p>

In [8]:
from sklearn.decomposition import TruncatedSVD

svd_transformer = TruncatedSVD(n_components=1000)

svd_transformer.fit(tfs)

print(sorted(svd_transformer.explained_variance_ratio_)[::-1][:30])

[0.04319143964864218, 0.018619648885290416, 0.015991826821457188, 0.015984996712032705, 0.013744333318752369, 0.011771580238264153, 0.0096502136679727582, 0.0095168074249578714, 0.0095050502041705634, 0.0094605420641273308, 0.0093174536752533427, 0.0092700425653136059, 0.0077114068911813705, 0.0069839976227606569, 0.0066546888854344659, 0.0064195285763082675, 0.0058853020300615594, 0.0053722779003033136, 0.0051581711906631012, 0.0049312579563956863, 0.0048675271120016935, 0.0047897360656975214, 0.0044624928650002895, 0.0043399578651272148, 0.0042969022436024646, 0.0041605617284113843, 0.0040394806461863204, 0.0039137333696623025, 0.0038585289999962837, 0.0038298284368970051]


<p>Agora vamos manter as dimensões até que a variância acumulada seja maior ou igual a 0.50.</p>

In [9]:
cummulative_variance = 0.0
k = 0
for var in sorted(svd_transformer.explained_variance_ratio_)[::-1]:
    cummulative_variance += var
    if cummulative_variance >= 0.5:
        break
    else:
        k += 1
        
print(k)

143


<p>Transformarmos novamente, mas desta vez com o número de k componentes que obtemos anteriormente.</p>

In [11]:
svd_transformer = TruncatedSVD(n_components=k)
svd_data = svd_transformer.fit_transform(tfs)
print(sorted(svd_transformer.explained_variance_ratio_)[::-1])

[0.043191439648672884, 0.018619648883417345, 0.015991826858359582, 0.015984996739453274, 0.013744333249896268, 0.011771579960380696, 0.0096502135927737202, 0.0095168071742513065, 0.0095050503528928427, 0.0094605422360137161, 0.009317453944640482, 0.0092700426922784143, 0.0077114076961383978, 0.0069839949092168879, 0.0066546898302034321, 0.0064195263269203598, 0.0058852936980912296, 0.0053722793434196623, 0.0051581705034021499, 0.0049312538165894974, 0.0048675261535410042, 0.004789732565309881, 0.0044624842454884517, 0.0043399501248714599, 0.0042969006188773064, 0.00416055574593406, 0.0040394733566841579, 0.0039137298223758969, 0.0038585318797753182, 0.0038298140891522641, 0.0037136338808673257, 0.0036634048773248437, 0.0035824588636568515, 0.0035617299738693034, 0.0035351647924154865, 0.0034284465525109675, 0.0033450509316554542, 0.0032429644299145728, 0.0032341007248697803, 0.0031762097278202309, 0.003122680063865171, 0.0030422258444417227, 0.0029313655524463721, 0.0029183200989965912

In [12]:
print(svd_data.shape)

(2355, 143)
